# COSADAMA - BASIC SQL WEEK4-2
 
작성자: 정찬, 이정윤   
참고자료: SQL/SB(MySQL)기본부터 파이썬/데이터분석 활용까지! (잔재미코딩), PyMySQL 모듈 이해 및 실습 (잔재미 코딩 블로그)

# 목차

1. PyMySQL로 xlsx(csv)파일 db에 넣기 - 파일의 크기가 작을 때
    
    
2. cmd로 txt파일 db에 넣기 - 파일의 크기가 클 때   
      2.1 mysql 환경변수 설정   
      2.2 cmd 환경에서 데이터를 db에 적재하기   
       
       
3. Pandas library와 PyMySQL 



# 1. PyMySQL로 xlsx(csv)파일 db에 넣기
앞서 2주차 과제를 하면서 workbench에서 xlsx(csv) 파일을 db에 넣는 방법을 배웠습니다. 
그렇다면, 이번엔 PyMySQL을 사용해서 xlsx(csv) 파일을 바로 db에 넣는 방법을 알아보겠습니다. csv나 xlsx 형식으로 raw data를 제공하는 경우가 많으니, 이 방법도 알면 보다 수월한 작업을 진행할 수 있을테죠!

방법은 여러가지가 있는데, 본 교안에서는 csv와 xlsx 모두 사용할 수 있는 방법 한가지를 소개하겠습니다. 

> 다른 방법이 궁금하다면 아래 블로그를 참고해보세요!  
1. [csv to mysql, mysql to csv (동영상 강의 있음)](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=oralol&logNo=221994582251)
2. [`mysql for excel` 다운받아서 하는 방법](https://zionh.tistory.com/44)

* 교안 참고 블로그: https://juun42.tistory.com/23

## 1.1 pandas 활용해서 xlsx(csv) 가져오기

기존에 Python에서 xlsx(csv)파일을 사용하듯 pandas를 활용해서 데이터를 가져옵니다. 

In [ ]:
import pandas as pd

data = pd.read_excel("data.xlsx")
data

In [26]:
'''csv 코드 (이외는 xlsx과 똑같음!)
import pandas as pd

data = pd.read_csv("data.csv", encoding='utf-8')
data
'''

,Kelly,Female,First
0,Tim,Male,Second
1,Yun,Female,Third


## 1.2 DB 만들어주기

기존에 있던 DB를 사용한다면 해당 부분은 진행하지 않아도 됩니다. 

In [8]:
### DB만들기

#1
import pymysql

#2. connect
db = pymysql.connect(host = 'localhost', port = 3306, user='root', passwd = '********', charset='utf8')

#3. curosr 사용하기
cursor = db.cursor() 

#4. sql 구문 (DB만들어주기) 만들기
sql = "CREATE DATABASE new_database" 

#5 cursor execute
cursor.execute(sql) 

#6. commit (최종 변경)
db.commit() 

#7. DB 닫아주기 
db.close() 

## 1.3 Table 만들기

DB와 같이 기존에 있는 DB의 Table을 사용한다면 해당 부분은 진행하지 않아도 됩니다.   
테이블을 새로 만든다면 넣어줄 데이터와 똑같은 형식 (컬럼, 데이터 타입)으로 지정하여 만들어주어야 합니다. 

In [29]:
#1
import pymysql

#2. connect
db = pymysql.connect(host='localhost',  port = 3306, user='root', password='********', db='new_database',charset='utf8') 

#3. curosr 사용하기
cursor = db.cursor() 

#4. sql 구문 (DB만들어주기) 만들기
my_table= "CREATE TABLE IF NOT EXISTS new_table(column1 VARCHAR(10) primary key, column2 VARCHAR(10), column3 varchar(10))" 

#5 cursor execute
cursor.execute(my_table)

#6. commit (최종 변경)
db.commit() 

#7. DB 닫아주기 
db.close() 

## 1.4 데이터 가져와서 DB에 넣어주기

DB에 데이터를 넣어주는 INSERT를 진행합니다. 
이때 for문을 사용해서 전체 데이터를 한번에 넣어줄 수 있습니다. (PyMySQL을 쓰는 보람이 느껴지는 부분!) 추가로, INSERT할 때 VALUE의 형식을 잘 지정해주어야 합니다!

In [30]:
#1
import pymysql

#2. connect
db = pymysql.connect(host='localhost',  port = 3306, user='root', password='********', db='new_database',charset='utf8') 

#3. curosr 사용하기
cursor = db.cursor() 

#4. sql 구문 (DB만들어주기) 만들기 + cursor execute
sql='INSERT INTO new_table(column1, column2, column3) VALUES(%s, %s, %s)'
for idx in range(len(data)):
    cursor.execute(sql, tuple(data.values[idx]))

#6. commit (최종 변경)
db.commit() 

#7. DB 닫아주기 
db.close() 

# 2. cmd로 txt파일 db에 넣기

여러분의 데이터베이스에 정말 엄청 큰 용량의 데이터가 계속해서 저장되고 있고, 이를 실시간으로 분석해야하는 상황을 가정해 봅시다. 컬럼의 개수는 100개가 넘어가고, row 수는 백만, 천만 단위입니다. 엑셀로는 100만건부터 표현이 불가능합니다. Pandas로 컬럼부터 삭제하고, 조건을 걸어서 선택하고, 혹시 모르니 따로 저장해두면서 변수마다 메모리를 엄청 잡아먹을 겁니다. 읽어오는 속도도 느리고, 무얼 어떻게 해야할지도 모르는 답답한 상황이죠. 그럴때! 우리는 cmd를 통해 database에 올려 두고 sql로 쉽게 원하는 데이터를 가져올 수 있습니다.

앞서 소개시켜드린 for문을 이용한 작업은 row가 1000건이 넘어가는 순간 급격히 느려지기 시작합니다. 이것이 for문의 한계입니다. numpy나 scrapy를 통해 느끼셨겠지만 for문을 통한 작업은 각각을 메모리에 올리고, 처리해야 하므로 느립니다. 한번에 하나씩 문자는 문자대로, 숫자는 숫자대로 따로따로 인식해 처리하죠. 몇만, 몇백만건을 처리해야하는 작업에는 너무 느려서 어울리지 않습니다. scrapy처럼 일단 필요한 부분을 다 가져와버리는 편이 효과적이죠. numpy처럼 자료를 array(쉽게 말해 행렬이나 텐서)로 처리하는 방법도 있습니다.

그래서 우리는 
- 엄청 큰 csv나 txt파일을 바로 db에 올리고, 
- PyMySQL로 원하는 조건을 쿼리로 선택해
- pandas로 바로 dataframe을 만들어 작업하는 환경을 만들어 볼겁니다.

나중에는 csv나 txt가 아니라 자동으로 실시간 데이터를 db에 적재하고, 이를 바로 PyMySQL로 가져와서 작업하실 수 있을겁니다.

단계는 크게 3개로 나뉩니다.
1. mysql 환경변수 설정
2. cmd 환경에서 데이터를 db에 적재
3. pymysql과 pandas 통해 원하는 데이터를 dataframe으로 가져오기
    - fetchone()
    - fetchmany()
    - fetchall()

## 2.1 mysql 환경변수 설정

[mysql 환경설정 방법](https://hoho325.tistory.com/163)으로 설명은 대체합니다. (Window10기준) 

## 2.2 cmd 환경에서 데이터를 db에 적재   

**0. [여기](http://data.seoul.go.kr/dataList/OA-12912/S/1/datasetView.do)에서 실습 데이터를 다운 받습니다.**

자세한 내용은 [이곳](https://github.com/Team-COSADAMA/Blog/blob/main/content/blog/MySQL-csv-import.md)을 참고해 주세요. 아직 블로그가 업로드 되지 않았는데, 이후 링크를 수정하겠습니다.

**1.  cmd 창을 엽니다.**


**2.  cmd에서 mysql 접속**     
`mysql -u root -p --local-infile` 이렇게 입력하고 엔터를 누른 후, mysql root 계정의 비밀번호를 입력합니다. 만약 안된다면 뒤의 --local-infile을 제거하고 접속해 줍니다.   


**3. local_infile=true**      
`> mysql show global variables like 'local_infile';` 앞부분의 > mysql은 이제 기본을 나올겁니다. 박스 안에 local_infile이 off로 처리되어 있다면 `mysql> set global local_infile=true;`를 입력하세요. 그리고 mysql 문법처럼 쿼리가 끝날 때 ; 를 꼭 써주셔야 한답니다! 이후 쿨하게 `exit`을 눌러서 나가줍니다.   


**4. db, table 만들기**    
cmd창을 다시 키고 `mysql --local_infile -u root -p`으로 접속한 뒤, `> mysql show global variables like 'local_infile';`를 다시 확인해 줍니다. on이 나오면 드디어 csv, txt 등 파일을 db에 넣어줄 거예요. 


**5. db, table 만들기 (db는 이미 만들었다면 pass!)**    
database를 선택하고(`use 원하는database`), table을 만들어 줍니다. 문법은 똑같으니 cmd창에서 진행해도 좋고, pymysql로 진행하셔도 됩니다.  


**6. data import**    
table에 데이터를 적재할겁니다. 

>LOAD DATA LOCAL INFILE "절대경로/BUS_STATION_BOARDING_MONTH_202109.txt"     
INTO TABLE dbName.tableName      
FIELDS TERMINATED BY "|"    
LINES TERMINATED BY "\n"    
IGNORE 1 ROWS     
-- (`column1`, `column2`, ...)    

    
이런 식으로 데이터를 넣어 줍니다. 백만건 기준으로 약 5~6초 정도 소요됩니다. 위의 for문으로는 3시간으로도 다 하지 못했습니다.

앞으로는 pymysql로 jupyter notebook이든, vscode든, pycharm이든 PyMySQL로 원하는 데이터를 가져올 수 있겠죠!

# 3. Pandas library와 PyMySQL 

PyMySQL을 사용하여 db를 조작할 때, select와 fetch를 사용하면 원하는 데이터를 python notebook에서 확인할 수 있습니다. fetch의 종류는 

- fetchone(): 한 행row만 가져옵니다.
- fetchmany(size= ): size만큼의 row를 가져옵니다.
- fetchall(): 모든 row를 가져옵니다.
    
등이 있습니다. 아래의 예시를 통해 확인해봅시다!

In [1]:
import pymysql

host_name = 'localhost'
host_port = 3306
username = 'root'
password = '*********'
database_name = 'your_db_name'

db = pymysql.connect(
     host=host_name,     # MySQL Server Address
     port=host_port,     # MySQL Server Port
     user=username,      # MySQL username
     passwd=password,    # password for MySQL username
     db=database_name,   # Database name
     charset='utf8'
 )
db
cursor = db.cursor()

In [2]:
# 전체 데이터를 한 행만 가져와 봅시다.
sql = "SELECT * FROM your_table_name"
cursor.execute(sql)
bus1 = cursor.fetchone()

꽤 오래 걸리죠? 제 데스크탑으로는 14.6초 정도 걸리네요. 더 걸리더라도 너무 놀라지 마세요:)

이렇게 하면 결과값은 list 형식으로 출력됩니다. 그렇다면, 우리가 익숙한 dataframe으로 db의 데이터를 가져오는 방법은 없을까요? 물론 있습니다. 바로 익숙한 pandas를 활용해서 말이죠!

사용법은 `변수명 = fetchall()` 와 같은 형식으로 저장한 후, `df명 = pd.dataframe(변수명)`으로 데이터프레임을 만들어 주면 끝! 아직 이해가 되지 않는다면 아래 예시를 활용해보세요.  

In [3]:
import pandas as pd

bus1 = pd.DataFrame(bus1)
bus1

,0
0,2021-09-01 00:00:00
1,100
2,100번(하계동~용산구청)
3,11428
4,한성여객종점
5,14
6,1
7,0000-00-00 00:00:00
8,1


In [4]:
type(bus1.columns)

pandas.core.indexes.range.RangeIndex

컬럼 이름이 rangeindex로 나오는군요 먼저 행열 전환을 해주고, 원래 컬럼명으로 바꿔서 저장해줍시다. pandas 조작법을 그대로 사용하면 됩니다!

In [11]:
import numpy as np

bus1 = np.transpose(bus1)

In [12]:
import pandas as pd

bus1 = pd.DataFrame(bus1)
bus1.columns = ['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수',
 '등록일자', 'id']

bus1

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자,id
0,2021-09-01,100,100번(하계동~용산구청),11428,한성여객종점,14,1,0000-00-00 00:00:00,1


승차총승객수가 1000이 넘는 엄청난 버스 노선이 있는지 해당 데이터만 가져와 봅시다.

In [15]:
# 승차총승객수가 1000이 넘는 엄청난 버스 노선이 있을까요?
sql = "SELECT * FROM your_table_name where 승차총승객수 >1000"
cursor.execute(sql)
bus1000_10 = cursor.fetchmany(size=10)

In [16]:
import pandas as pd

bus1000_10 = pd.DataFrame(bus1000_10)
bus1000_10.columns = ['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수',
 '등록일자', 'id']

bus1000_10

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자,id
0,2021-09-01,5413,5413번(시흥~고속터미널),21350,신림사거리.신림역,1028,429,0000-00-00 00:00:00,1028
1,2021-09-01,5627,5627번(노온사동~구로디지털단지역),17015,구로디지털단지역환승센터,1050,574,0000-00-00 00:00:00,3073
2,2021-09-01,7734,7734번(진관공영차고지~홍대입구역),14015,홍대입구역,1431,28,0000-00-00 00:00:00,3805
3,2021-09-01,6635,6635번(광명공영차고지~구로디지탈단지역),17014,구로디지털단지역환승센터,1220,1108,0000-00-00 00:00:00,4010
4,2021-09-01,3315,3315번(마천동~송파구민회관),24146,잠실역.롯데월드,1027,835,0000-00-00 00:00:00,4239
5,2021-09-01,143,143번(정릉~개포동),22019,고속터미널,1683,707,0000-00-00 00:00:00,4868
6,2021-09-01,9401,9401번(구미동~서울역),47638,낙생육교,1021,178,0000-00-00 00:00:00,8142
7,2021-09-01,9401,9401번(구미동~서울역),3164,순천향대학병원,1060,34,0000-00-00 00:00:00,8164
8,2021-09-01,110A,"""110번(A,고려대방향,정릉~정릉)""",3162,순천향대학병원,1170,336,0000-00-00 00:00:00,8322
9,2021-09-01,641,641번(문래동~양재동),19192,문래역,1039,193,0000-00-00 00:00:00,9778


승차총승객수가 1000이상인 데이터를 다 가져와 봅시다.

In [19]:
# 승차총승객수가 1000이 넘는 엄청난 버스 노선이 있을까요?
sql = "SELECT * FROM your_table_name where 승차총승객수 >1000"
cursor.execute(sql)
bus1000_all = cursor.fetchall()

In [20]:
import pandas as pd

bus1000_all = pd.DataFrame(bus1000_all)
bus1000_all.columns = ['사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수',
 '등록일자', 'id']

bus1000_all

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자,id
0,2021-09-01,5413,5413번(시흥~고속터미널),21350,신림사거리.신림역,1028,429,0000-00-00 00:00:00,1028
1,2021-09-01,5627,5627번(노온사동~구로디지털단지역),17015,구로디지털단지역환승센터,1050,574,0000-00-00 00:00:00,3073
2,2021-09-01,7734,7734번(진관공영차고지~홍대입구역),14015,홍대입구역,1431,28,0000-00-00 00:00:00,3805
3,2021-09-01,6635,6635번(광명공영차고지~구로디지탈단지역),17014,구로디지털단지역환승센터,1220,1108,0000-00-00 00:00:00,4010
4,2021-09-01,3315,3315번(마천동~송파구민회관),24146,잠실역.롯데월드,1027,835,0000-00-00 00:00:00,4239
...,...,...,...,...,...,...,...,...,...
2981,2021-09-28,강남06,강남06(세곡LH푸르지오아파트~코엑스),23408,수서역,1167,380,0000-00-00 00:00:00,1048338
2982,2021-09-28,강남02,강남02(강남구민체육관~양재역),23920,양재역.종합복지관,1058,153,0000-00-00 00:00:00,1048399
2983,2021-09-28,강남02,강남02(강남구민체육관~양재역),23323,매봉역,1486,86,0000-00-00 00:00:00,1048400
2984,2021-09-28,강동05,강동05(강동공영차고지~천호역),25162,천호역현대백화점.노동권익센터,1196,46,0000-00-00 00:00:00,1048511


database는 다르지만 아래와 같이 포매팅(%)을 통해 다양한 데이터를 한번에 가져올 수도 있습니다. 직접 코드를 실행해 보세요:)

In [46]:
# SELECT 
sql = "SELECT * FROM your_table_name WHERE 노선번호 REGEXP '^[강남]';"
cursor.execute(sql)

34008

In [ ]:
# DELETE
sql = "DELETE FROM your_table_name WHERE 노선번호 REGEXP '^[강남]';"
cursor.execute(sql)
db.commit()

In [ ]:
data = [['사용일자', 원하는_날짜], ['노선번호', 원하는_번호], ['노선명', 원하는_노선명], ['버스정류장ARS번호',원하는_번호], 
['역명', 원하는_역명], ['승차총승객수', 원하는_숫자], ['하차총승객수', 원하는_숫자], ['등록일자', 원하는_날짜], ['id', 원하는_id]]

# INSERT 
sql = """INSERT INTO your_table_name('사용일자', '노선번호', '노선명', '버스정류장ARS번호', '역명', '승차총승객수', '하차총승객수',
 '등록일자', 'id') VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
 """
 
cursor.executemany(sql, data)
db.commit()

In [49]:
data = [['노선명', 10000], ['노선명', 10000]]

# UPDATE 
sql = "UPDATE your_table_name SET `노선명` = %s WHERE `id` < %s;"
cursor.executemany(sql, data)
db.commit()